In [1]:
import pandapower.networks as pn
import os
import pandapower as pp
import polars as pl
from polars import col as c
from shapely import  from_geojson
from plotly.graph_objects import Figure
import plotly.graph_objs as go
from general_function import pl_to_dict, build_non_existing_dirs

from data_display.grid_plotting import plot_grid_from_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from polars_function import list_to_list_of_tuple


os.chdir(os.getcwd().replace("/src", ""))


In [2]:
net = pn.case118()

2025-06-19 14:11:38 B01BPC06 pandapower.convert_format[1444769] INFO These dtypes could not be corrected: {'trafo': ['tap_neutral', 'tap_min', 'tap_max', 'tap_pos']}


In [3]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source


In [4]:
net["sgen"] = net.gen[["name", "bus", "p_mw", "sn_mva", "scaling", "in_service", "type"]]
net["sgen"]["q_mvar"] = 0
net["sgen"]["current_source"] = True
net["gen"] = net["gen"].drop(net["gen"].index)

In [5]:
# remove all shunts
net.shunt = net.shunt.iloc[0:0]

In [6]:
# remove all polynomial cost entries
net.poly_cost = net.poly_cost.iloc[0:0]

In [7]:
net

This pandapower network includes the following parameter tables:
   - bus (118 elements)
   - load (99 elements)
   - sgen (53 elements)
   - ext_grid (1 element)
   - line (173 elements)
   - trafo (13 elements)

In [8]:
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,min_vm_pu,geo
0,1,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-2.2753708781, 2.8543413351],..."
1,2,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-2.9368186836, 2.2121792656],..."
2,3,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-1.8344312496, 1.7094451782],..."
3,4,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-0.8886268958, 1.5532705585],..."
4,5,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-0.9632829393, 0.694729907], ..."
...,...,...,...,...,...,...,...,...
113,114,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [2.2630604919, -2.7868829987],..."
114,115,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [3.1644478736, -2.3853607175],..."
115,116,345.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-4.1528539003, -4.9348933434]..."
116,117,138.0,b,1.0,True,1.06,0.94,"{""coordinates"": [-3.5172125372, 1.7774067282],..."


In [9]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent,geo
0,None,None,0,1,1.0,5.770332,19.024956,353.789081,0.0,41.418606,1.0,1,ol,True,100.0,NaN
1,None,None,0,2,1.0,2.456676,8.074656,150.708577,0.0,41.418606,1.0,1,ol,True,100.0,NaN
2,None,None,3,4,1.0,0.335174,1.519711,29.250278,0.0,41.418606,1.0,1,ol,True,100.0,NaN
3,None,None,2,4,1.0,4.589604,20.567520,395.575193,0.0,41.418606,1.0,1,ol,True,100.0,NaN
4,None,None,4,5,1.0,2.266236,10.283760,198.623319,0.0,41.418606,1.0,1,ol,True,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,None,None,26,114,1.0,3.123216,14.111604,274.674042,0.0,41.418606,1.0,1,ol,True,100.0,NaN
169,None,None,113,114,1.0,0.438012,1.980576,38.443223,0.0,41.418606,1.0,1,ol,True,100.0,NaN
170,None,None,11,116,1.0,6.265476,26.661600,498.647602,0.0,41.418606,1.0,1,ol,True,100.0,NaN
171,None,None,74,117,1.0,2.761380,9.160164,166.865874,0.0,41.418606,1.0,1,ol,True,100.0,NaN


In [10]:
bus: pl.DataFrame = pl.from_pandas(net["bus"]).with_columns(
    c("geo").map_elements(lambda x: list(from_geojson(x).coords)[0], return_dtype=pl.List(pl.Float64)),
).with_row_index(name="bus_id")


line: pl.DataFrame = pl.from_pandas(net["line"])


In [11]:
pl.from_pandas(net["trafo"])

name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,i0_percent,shift_degree,tap_side,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,parallel,df,in_service,max_loading_percent,tap_changer_type
str,str,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,u32,f64,bool,f64,str
null,null,7,4,9900.0,345.0,138.0,264.33,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,1.5,null,-1.0,1,1.0,true,100.0,"""Ratio"""
null,null,25,24,9900.0,345.0,138.0,378.18,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,4.0,null,-1.0,1,1.0,true,100.0,"""Ratio"""
null,null,29,16,9900.0,345.0,138.0,384.12,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,4.0,null,-1.0,1,1.0,true,100.0,"""Ratio"""
null,null,37,36,9900.0,345.0,138.0,371.25,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,6.5,null,-1.0,1,1.0,true,100.0,"""Ratio"""
null,null,62,58,9900.0,345.0,138.0,382.14,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,4.0,null,-1.0,1,1.0,true,100.0,"""Ratio"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,67,68,9900.0,161.0,138.0,366.3,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,6.5,null,-1.0,1,1.0,true,100.0,"""Ratio"""
null,null,80,67,9900.0,345.0,161.0,200.729061,17.325,0.0,-0.816162,0.0,null,null,null,null,null,null,null,1,1.0,true,100.0,null
null,null,80,79,9900.0,345.0,138.0,366.3,0.0,0.0,0.0,0.0,"""hv""",0.0,null,null,6.5,null,-1.0,1,1.0,true,100.0,"""Ratio"""


In [12]:
trafo: pl.DataFrame = pl.from_pandas(net["trafo"]).drop("name").with_row_index(name="name").with_columns(
    ("Trafo_" + c("name").cast(pl.Utf8)).alias("name")
)



In [13]:

switch = line.rename({"name": "line_name"}).with_row_index(name="name")\
.select(
    c("line_name"),
    c("to_bus").alias("element"),
    pl.lit("b").alias("et"),
    pl.lit("LBS").alias("type"),
    pl.lit(True).alias("closed"),
    c("name"),
    pl.lit(0.0).alias("z_ohm"),
    pl.lit(100.0).alias("in_ka"),
 
    
).with_row_index(name="bus", offset=bus.height)\
.with_columns(
    ("switch " + c("name").cast(pl.Utf8)).alias("name"),
)
new_bus_mapping = pl_to_dict(switch["line_name", "bus"])

line = line.drop("name").with_row_index(name="name").with_columns(
    ("Branche_" + c("name").cast(pl.Utf8)).alias("name")
)

bus = pl.concat([
    bus,
    switch.select(
    c("bus").alias("bus_id"),
    ("Bus " + c("bus").cast(pl.Utf8)).alias("name"),
    pl.lit(20).alias("vn_kv"),
    pl.lit("b").alias("type"),
    pl.lit("CIGRE_MV").alias("zone"),
    pl.lit(True).alias("in_service"),
)
], how="diagonal_relaxed")



net["bus"] = bus.to_pandas()
net["switch"] = switch.drop("line_name").to_pandas()
net["line"] = line.to_pandas()
net["trafo"] = trafo.to_pandas()

net["trafo"]["shift_degree"] = 0


build_non_existing_dirs(".cache/input_data")
pp.to_pickle(net, ".cache/input_data/mv_example.p")

In [14]:
net["line"]

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent,geo
0,Branche_0,None,0,1,1.0,5.770332,19.024956,353.789081,0.0,41.418606,1.0,1,ol,True,100.0,NaN
1,Branche_1,None,0,2,1.0,2.456676,8.074656,150.708577,0.0,41.418606,1.0,1,ol,True,100.0,NaN
2,Branche_2,None,3,4,1.0,0.335174,1.519711,29.250278,0.0,41.418606,1.0,1,ol,True,100.0,NaN
3,Branche_3,None,2,4,1.0,4.589604,20.567520,395.575193,0.0,41.418606,1.0,1,ol,True,100.0,NaN
4,Branche_4,None,4,5,1.0,2.266236,10.283760,198.623319,0.0,41.418606,1.0,1,ol,True,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Branche_168,None,26,114,1.0,3.123216,14.111604,274.674042,0.0,41.418606,1.0,1,ol,True,100.0,NaN
169,Branche_169,None,113,114,1.0,0.438012,1.980576,38.443223,0.0,41.418606,1.0,1,ol,True,100.0,NaN
170,Branche_170,None,11,116,1.0,6.265476,26.661600,498.647602,0.0,41.418606,1.0,1,ol,True,100.0,NaN
171,Branche_171,None,74,117,1.0,2.761380,9.160164,166.865874,0.0,41.418606,1.0,1,ol,True,100.0,NaN


In [15]:

# 1) load the example network
net = pp.from_pickle(".cache/input_data/mv_example.p")

# 2) convert -> returns a dict
schema = pandapower_to_dig_a_plan_schema(net)
node_df = schema["node_data"]
edge_df = schema["edge_data"]
print(edge_df)
print(node_df)
# quick check
print(type(node_df), node_df.shape)   # should be polars.DataFrame
print(type(edge_df), edge_df.shape)

shape: (359, 12)
┌─────────┬───────────┬───────────┬────────────┬───┬────────┬──────────┬──────┬───────────┐
│ edge_id ┆ u_of_edge ┆ v_of_edge ┆ eq_fk      ┆ … ┆ type   ┆ i_base   ┆ g_pu ┆ n_transfo │
│ ---     ┆ ---       ┆ ---       ┆ ---        ┆   ┆ ---    ┆ ---      ┆ ---  ┆ ---       │
│ u32     ┆ i32       ┆ i32       ┆ str        ┆   ┆ str    ┆ f64      ┆ f64  ┆ f64       │
╞═════════╪═══════════╪═══════════╪════════════╪═══╪════════╪══════════╪══════╪═══════════╡
│ 0       ┆ 0         ┆ 1         ┆ Branche_0  ┆ … ┆ branch ┆ 4.183698 ┆ null ┆ null      │
│ 1       ┆ 0         ┆ 2         ┆ Branche_1  ┆ … ┆ branch ┆ 4.183698 ┆ null ┆ null      │
│ 2       ┆ 3         ┆ 4         ┆ Branche_2  ┆ … ┆ branch ┆ 4.183698 ┆ null ┆ null      │
│ 3       ┆ 2         ┆ 4         ┆ Branche_3  ┆ … ┆ branch ┆ 4.183698 ┆ null ┆ null      │
│ 4       ┆ 4         ┆ 5         ┆ Branche_4  ┆ … ┆ branch ┆ 4.183698 ┆ null ┆ null      │
│ …       ┆ …         ┆ …         ┆ …          ┆ … ┆ …      ┆ …